In [11]:

import os
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import chardet

In [6]:
#train = pd.read_csv(os.path.join(settings.PROCESSED_DIR, "train.csv"))
data_dir = './data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/val'
test_dir = data_dir + '/test'

In [21]:
with open('data/Animated_Tv_Series.csv', 'rb') as f:
    result = chardet.detect(f.read())
types_dict = {'A': int, 'B': float}
types_dict.update({col: str for col in col_names if col not in types_dict})
pd.read_csv('file.csv', dtype=types_dict)
df = pd.read_csv('data/Animated_Tv_Series.csv', encoding=result['encoding'])
df.head()
df.shape

(1816, 10)

In [35]:
# df.iloc[1813]
df.dtypes

Id                    int64
Title                object
Episodes            float64
Year                 object
Original channel     object
American company     object
Note                 object
Technique            object
IMDb                float64
Google users         object
dtype: object

In [36]:
df[df.isnull().any(axis=1)]

,Id,Title,Episodes,Year,Original channel,American company,Note,Technique,IMDb,Google users
0,1,2 Stupid Dogs,26.0,1993-1995,TBS,Hanna-Barbera,NaN,Traditional,7.2,91%
1,2,3-2-1 Penguins!,27.0,2000-2008,Direct to video,Big Idea Productions,NaN,CGI,6.5,91%
2,3,3-South,13.0,2002-2003,MTV,Warner Bros. AnimationMTV Animation,NaN,Traditional,8.0,87%
3,4,3Below: Tales of Arcadia,28.0,2018-2019,Netflix,DreamWorks Animation Television,NaN,CGI,7.6,77%
5,6,The 7D,44.0,2014-2016,Disney XD,Disney Television Animation,NaN,Traditional,5.7,85%
...,...,...,...,...,...,...,...,...,...,...
1809,1810,The Zeta Project,26.0,2001-2002,Kids' WB,Warner Bros. Animation,NaN,Traditional,7.0,93%
1810,1811,Zevo-3,26.0,2010-2011,Nicktoons,MoonScoop Entertainment,NaN,Traditional,5.8,93%
1813,1814,Zombie College,12.0,2000-2001,NaN,icebox.com,NaN,Flash,6.8,62%
1814,1815,Zootopia+,6.0,2022,Disney+,Walt Disney Animation Studios,NaN,CGI,6.8,74%


In [47]:
#Preprocessing/Cleaning
techn = df['Technique'].unique()
arr_ = list(map(lambda x: len(x), techn))  # remark: py3 would work different here
x = techn[np.argsort(arr_)]
print(x)

['CGI' 'Flash' 'CGI/Flash' 'Traditional' 'Stop-Motion' 'Stop-motion'
 'CGI/Live-action' 'Traditional/CGI' 'CGI/Live-Action' 'Flash/Live-action'
 'Traditional/Flash' 'Flash/Live action' 'Flash/Live-Action'
 'Flash/Traditional' 'Live-Action/Flash' 'Flash/Traditional/CGI'
 'CGI/Flash/Live-action' 'CGI/Traditional/Flash' 'Stop-Motion/CGI/Flash'
 'Stop-Motion/Live-Action' 'Live-Action/Traditional'
 'Traditional/Live-action' 'Stop-motion/Live-action'
 'Traditional/Live-Action' 'Traditional/CGI/Stop-Motion'
 'Flash/Traditional/Live-Action' 'Flash/Live-Action/Traditional/CGI'
 'CGI/Stop-Motion/Flash/Traditional' 'Traditional/Stop-Motion/CGI/Flash'
 'Traditional/Live-Action/Flash/CGI' 'CGI/Flash/Stop-motion/Traditional'
 'Traditional/Flash/CGI/Stop-Motion' 'CGI/Flash/Stop-Motion/Live-Action'
 'Traditional (season 1)/Flash (season 2)'
 'Traditional/CGI/Stop-Motion/Live-Action'
 'Traditional (season 1)/Flash (season 2-3)'
 'Traditional (seasons 1-9)/Flash (season 10)'
 'CGI/Flash/Stop-Motion/Trad

In [ ]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [89]:
one_hot_cols_technique = ["CGI","Flash","Traditional","Stop","Live","Digital"]

for col in one_hot_cols_technique:
    df[f"{col}"] = np.where(df['Technique'].str.contains(f"{col}"),1,0)
   

In [95]:
df = df.drop(['StopMotion', 'LiveAction'], axis=1)

In [96]:
df

,Id,Title,Episodes,Year,Original channel,American company,Note,Technique,IMDb,Google users,CGI,Flash,Traditional,Digital,Stop,Live
0,1,2 Stupid Dogs,26.0,1993-1995,TBS,Hanna-Barbera,NaN,Traditional,7.2,91%,0,0,1,0,0,0
1,2,3-2-1 Penguins!,27.0,2000-2008,Direct to video,Big Idea Productions,NaN,CGI,6.5,91%,1,0,0,0,0,0
2,3,3-South,13.0,2002-2003,MTV,Warner Bros. AnimationMTV Animation,NaN,Traditional,8.0,87%,0,0,1,0,0,0
3,4,3Below: Tales of Arcadia,28.0,2018-2019,Netflix,DreamWorks Animation Television,NaN,CGI,7.6,77%,1,0,0,0,0,0
4,5,3rd & Bird,51.0,2008-2010,Playhouse Disney,Little Airplane Productions,British co-production,Flash,7.8,92%,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1811,1812,The ZhuZhus,26.0,2016-2017,Disney Channel,Cepia LLC,Canadian co-production,Flash,5.5,72%,0,1,0,0,0,0
1812,1813,Zoboomafoo,65.0,1999-2001,PBS Kids,Maryland Public Television,Canadian co-production,Stop-motion/Live-action,7.6,95%,0,0,0,0,1,1
1813,1814,Zombie College,12.0,2000-2001,NaN,icebox.com,NaN,Flash,6.8,62%,0,1,0,0,0,0
1814,1815,Zootopia+,6.0,2022,Disney+,Walt Disney Animation Studios,NaN,CGI,6.8,74%,1,0,0,0,0,0
